## Import Packages

In [16]:
import pandas as pd
#import boto3
import pickle
import json
import numpy as np
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Load data from S3

In [7]:
bucket = 'csp554-project-crashes'
subfolder = 'Crashes/0/'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
try:
    del(data)
except:
    pass
for f in contents:
    print(f['Key'])
    response = conn.get_object(Bucket=bucket, Key=f['Key'])
    body = response['Body']
    body = body.read()
    body = body.decode('utf-8')
    df = json.loads(body)
    df= pd.DataFrame(df)
    
    if 'data' in locals():
        data = pd.concat([data, df])
    else:
        data = df.copy()
data2 = data.copy()

Crashes/0/19.json
Crashes/0/29.json
Crashes/0/39.json
Crashes/0/49.json
Crashes/0/59.json
Crashes/0/62.json
Crashes/0/9.json


In [6]:
import glob
import pandas as pd

In [105]:
try:
    del(data)
except:
    pass
for file in glob.glob("C:\MS in Data Science\IIT\Courses\CSP 554 - Big Data\Project\Datasets\Crashes\*.json"):
    df= pd.read_json(file)
    
    if 'data' in locals():
        data = pd.concat([data, df])
    else:
        data = df.copy()

In [111]:
data.groupby(["lighting_condition"])['injuries_total'].count()

lighting_condition
0    13027
1    18231
Name: injuries_total, dtype: int64

In [8]:
data.shape

(31258, 48)

## Drop Unnecessary Variables

In [106]:
data = data.fillna(0)
data = data.drop(['crash_record_id', 'crash_date', 'date_police_notified', 'prim_contributory_cause', 'sec_contributory_cause',
                 'street_no', 'street_name', "injuries_fatal", "injuries_incapacitating", "injuries_reported_not_evident",
                  "injuries_no_indication", "injuries_unknown", "crash_month", "latitude", "longitude", "private_property_i",
                  "crash_date_est_i", "photos_taken_i", "statements_taken_i", "work_zone_i", "workers_present_i", "dooring_i",
                  "rd_no", "lane_cnt", "work_zone_type", "intersection_related_i", "hit_and_run_i",
                  "first_crash_type", "alignment", "road_defect", "street_direction", "beat_of_occurrence", 
                  "injuries_non_incapacitating"
                 ], axis=1)

In [40]:
data.columns

Index(['posted_speed_limit', 'traffic_control_device', 'device_condition',
       'weather_condition', 'lighting_condition', 'trafficway_type',
       'roadway_surface_cond', 'report_type', 'crash_type', 'damage',
       'num_units', 'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week'],
      dtype='object')

## Data Wrangling

In [107]:
data.loc[:,"traffic_control_device"] = np.where(data.loc[:,"traffic_control_device"]!="NO CONTROLS", 1, 0)
data.loc[:,"device_condition"] = np.where(data.loc[:,"device_condition"]=="FUNCTIONING PROPERLY", 1, 0)
data.loc[:,"weather_condition"] = np.where(data.loc[:,"weather_condition"].isin(["CLEAR", "CLOUDY/OVERCAST"]), 1, 0)
data.loc[:,"lighting_condition"] = np.where(data.loc[:,"lighting_condition"]=="DAYLIGHT", 1, 0)
data.loc[:,"roadway_surface_cond"] = np.where(data.loc[:,"roadway_surface_cond"]=="DRY", 1, 0)
data.loc[:,"report_type"] = np.where(data.loc[:,"report_type"]=="ON SCENE", 1, 0)
data.loc[:,"crash_type"] = np.where(data.loc[:,"crash_type"]=="NO INJURY / DRIVE AWAY", 1, 0)
data.loc[:,"num_units"] = data.loc[:,"num_units"].astype(int)
data.loc[:,"num_units"] = np.where(data.loc[:,"num_units"]>=3, 3, data.loc[:,"num_units"])
data.loc[:,"most_severe_injury"] = np.where(data.loc[:,"most_severe_injury"]!="NO INDICATION OF INJURY", 1, 0)
data.loc[:,"injuries_total"] = np.where(data.loc[:,"injuries_total"]>=4, 4, data.loc[:,"injuries_total"])

# to be One hot encoded
data.loc[:,"trafficway_type"] = np.where(~data.loc[:,"trafficway_type"].isin(["DIVIDED - W/MEDIAN (NOT RAISED)", "NOT DIVIDED"]), "Other", data.loc[:,"trafficway_type"])
data.loc[:,"crash_hour"] = data.loc[:,"crash_hour"].astype(str)
data.loc[:,"crash_day_of_week"] = data.loc[:,"crash_day_of_week"].astype(str)

# Target variable
data.loc[:,"damage"] = np.where(data.loc[:,"damage"]=="OVER $1,500", 1, 0)

## Correlation matrix

In [108]:
numeric_cols = ["posted_speed_limit", "num_units", "injuries_total"]
for col in data.columns:
    if col not in numeric_cols:
        data[col] = data[col].astype('object')

In [109]:
# None of the numeric columns were correlated
corr_df = data[numeric_cols].corr()
print(corr_df)
for col in corr_df:
    for index in corr_df:
        if index!= col:
            if corr_df.loc[index, col] >= 0.5:
                print(index, col)

                    posted_speed_limit  num_units  injuries_total
posted_speed_limit            1.000000   0.045485        0.084128
num_units                     0.045485   1.000000        0.085674
injuries_total                0.084128   0.085674        1.000000


In [48]:
data.columns

Index(['posted_speed_limit', 'traffic_control_device', 'device_condition',
       'weather_condition', 'lighting_condition', 'trafficway_type',
       'roadway_surface_cond', 'report_type', 'crash_type', 'damage',
       'num_units', 'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week'],
      dtype='object')

In [113]:
for col1 in data.select_dtypes(include=['object']).columns:
    for col2 in data.select_dtypes(include=['object']).columns:
        if col1!=col2:
            contingency_table = pd.crosstab(data[col1].astype(str), data[col2].astype(str))
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            if p < 0.05:
                print('P-value:', col1, col2, p)

P-value: weather_condition crash_type 0.00035108871121433644
P-value: weather_condition most_severe_injury 1.2526425696981233e-08
P-value: weather_condition crash_hour 2.8751404005435644e-61
P-value: weather_condition crash_day_of_week 8.719522063347892e-99
P-value: crash_type weather_condition 0.0003510887112143366
P-value: crash_type damage 0.0
P-value: crash_type most_severe_injury 0.0
P-value: crash_type crash_hour 4.150083036070965e-219
P-value: crash_type crash_day_of_week 9.410984099507015e-26
P-value: damage crash_type 0.0
P-value: damage most_severe_injury 2.7945883212360286e-42
P-value: damage crash_hour 2.6751452909904767e-52
P-value: damage crash_day_of_week 2.0230674610185294e-09
P-value: most_severe_injury weather_condition 1.2526425696981233e-08
P-value: most_severe_injury crash_type 0.0
P-value: most_severe_injury damage 2.7945883212360286e-42
P-value: most_severe_injury crash_hour 2.0467950262219302e-16
P-value: most_severe_injury crash_day_of_week 0.010073591838760831

In [112]:
data = data.drop(["traffic_control_device", "device_condition", "trafficway_type", 
                  "report_type", "roadway_surface_cond", "lighting_condition"],axis=1)

In [102]:
data.head()

,posted_speed_limit,weather_condition,lighting_condition,roadway_surface_cond,crash_type,damage,num_units,most_severe_injury,injuries_total,crash_hour,crash_day_of_week
0,30,0,1,0,1,0,2,0,0.0,16,4
1,30,1,1,1,1,0,2,0,0.0,15,4
2,30,1,1,1,1,1,2,0,0.0,15,4
3,30,0,1,0,1,0,2,0,0.0,15,4
4,25,1,1,1,0,1,3,0,0.0,15,4


## One Hot Encoding

In [114]:

data = pd.get_dummies(data, columns=['crash_hour', 'crash_day_of_week'] )


In [121]:
for col in data.columns:
    data.loc[:,col] = data.loc[:,col].astype(int)

In [115]:
data.shape

(31258, 38)

In [122]:
y = "damage"
train, test = train_test_split(data, test_size=0.2, random_state = 100, stratify=data[y])
x_train = train.drop(y, axis=1)
y_train = train[y]
x_test = test.drop(y, axis=1)
y_test = test[y]

## Logistic Regression

In [123]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg = logreg.fit(x_train, y_train)
yhat_logit_test = logreg.predict(x_test)
yhat_logit_train = logreg.predict(x_train)
print(confusion_matrix(y_test, yhat_logit_test))
print("training error: ", accuracy_score(y_train, yhat_logit_train))
print("testing error: ", accuracy_score(y_test, yhat_logit_test))

# Random Forests (Bagging model)
# Hyperparameter tuning using Grid Search
param_grid = {"criterion": ['gini', 'entropy'],
             "n_estimators": [200, 400, 600],
              "min_samples_leaf": [40],
              "max_depth": [3,5,7]
                #"max_features": [None, 'auto']
              }


## Random Forests

In [126]:
rf = RandomForestClassifier(n_jobs=8)
gs = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid,
    cv=3,
    refit=True,
    verbose=True,
    n_jobs=8)
gs.fit(x_train, y_train)
print(gs.best_params_)

rf = RandomForestClassifier(max_depth=3, 
                            random_state=4, 
                            min_samples_leaf=40, 
                            n_jobs=4,
                            n_estimators=200,
                            criterion='gini'
                           )
rf.fit(x_train.values, y_train.values)
yhat_rf = rf.predict(x_test)
print("Testing Error: ", accuracy_score(y_test, yhat_rf))
yhat_rf_train = rf.predict(x_train)
print("Training Error: ", accuracy_score(y_train, yhat_rf_train))

Fitting 3 folds for each of 18 candidates, totalling 54 fits


{'criterion': 'gini',
 'max_depth': 3,
 'min_samples_leaf': 40,
 'n_estimators': 200}